**This is the simplified Opensantions data in csv format**

In [1]:
#import packages
import requests
import pandas as pd
import numpy as np
import datetime
import os
import re

# Retrieval of the data

In [2]:
#I can access the data via pandas very easily, output already in tabular form
#data = pd.read_csv('https://data.opensanctions.org/datasets/20240121/us_ofac_sdn/targets.simple.csv')
#data

In [3]:
#create urls for all relevant dates
date_list = pd.date_range(start='20210801',end='20231231',freq='D').strftime('%Y%m%d')
date_list

Index(['20210801', '20210802', '20210803', '20210804', '20210805', '20210806',
       '20210807', '20210808', '20210809', '20210810',
       ...
       '20231222', '20231223', '20231224', '20231225', '20231226', '20231227',
       '20231228', '20231229', '20231230', '20231231'],
      dtype='object', length=883)

In [4]:
#get all the urls for all the dates we need the data for
#make each day an individual list so we can compare and match them
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/eu_sanctions_map/targets.simple.csv'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20210801/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210802/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210803/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210804/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210805/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210806/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210807/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210808/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210809/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210810/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210811/eu_sanctions_map/targets.simple.csv', 'https://data.opensanctions.org

#first check for missing days
for site in websites:
    try:
        response = requests.get(site)
        response.raise_for_status()  # Raises an HTTPError for bad response
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {site}: {e}")
        continue

In [5]:
#now loop over the batches to get all the data and append the dataframes to one another
#this gives us a list of dataframes
#the loop must contain a date identifyer as a new column so each dataframe is marked with the retrieval date that is then needed to match the different days against each other
entities_list = []
date_pattern = r'/datasets/(\d{8})/'
for site in websites:
    response = requests.get(site)
    if response.status_code != 200:
        continue
    data = pd.read_csv(site, low_memory=False)
    match = re.search(date_pattern, site) #extract the date from the url
    if match:
        date = match.group(1)
        data['date_stamp'] = date #set an individual date stamp for each dataframe matching the url date
        #ata =  data.set_index('date_stamp') #set as an index
    entities_list.append(data)
res = pd.concat(entities_list)  # concatenate list of dataframes

In [6]:
res

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,date_stamp,last_change
0,eu-sancmap-175cbd95e16f55350037259bffd0db5c703...,Vessel,CHON MYONG 1 Other information: DPRK oil tanke...,NaN,NaN,NaN,NaN,8712362,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN
1,eu-sancmap-22465b7c323f62f811f983b17e05644658b...,LegalEntity,MIN NING DE YOU 078,NaN,NaN,NaN,NaN,NaN,M/V MIN NING DE YOU engaged in a ship-to-ship ...,NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN
2,eu-sancmap-2e29d7cedca1b3e07b7809dcdcf0525250d...,Vessel,NEW REGENT,NaN,NaN,NaN,NaN,8312497,The M/V NEW REGENT engaged in a ship-to-ship t...,NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN
3,eu-sancmap-5f9d3445c8efce7a990a925754e029aca21...,Person,Ahmoudou AG ASRIW (alias: a) Amadou Ag Isriw; ...,NaN,NaN,NaN,NaN,NaN,Ahmoudou Ag Asriw is a senior commander of the...,NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN
4,eu-sancmap-66f96b24db9833c702bb3560ac4bb3ca4ad...,Vessel,BILLIONS NO. 18,NaN,NaN,NaN,NaN,9191773,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,eu-sancmap-8fbbf594e557333e4280ff018b11f837fd8...,Vessel,WAN HENG 11,NaN,NaN,NaN,NaN,8791667,M/V WAN HENG 11 engaged in a ship-to-ship tran...,NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-12-31T18:48:01,20231231,2023-04-20T12:14:15
95,eu-sancmap-a1a2a200b111d7825a25201865cac570e9b...,LegalEntity,Tsargrad,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2023-07-04T12:03:21,2023-12-31T18:48:01,20231231,2023-07-04T12:03:21
96,eu-sancmap-dd03ba813d7c14cc9570ed2f49627d63908...,Vessel,FAN KE,NaN,NaN,NaN,NaN,8914934,"M/V FAN KE loaded DPRK coal at Nampo, DPRK, in...",NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-12-31T18:48:01,20231231,2023-04-20T12:14:15
97,eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...,Vessel,PETREL 8,NaN,NaN,NaN,NaN,620233000;9562233,"""""",NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-12-31T18:48:01,20231231,2023-04-20T12:14:15


# Match the dates against each other

In [7]:
#now match the different dates against each other and return an indicator per row for a new listing or delisting
# Sort the DataFrame based on date_stamp and id
res.sort_values(by=['id', 'date_stamp'], inplace=True)

# Identify new entries and deletions based on the index and columns you want to compare
new_entries = ~res.duplicated(subset=['id'], keep='first')
deletions = ~res.duplicated(subset=['id'], keep='last')

# Identify unchanged rows by checking for duplicates based on the id and date_stamp
unchanged_rows = ~new_entries & ~deletions

# Set the values of new_entry, deletion, and unchanged columns
res['new_entry'] = new_entries
res['deletion'] = deletions
res['unchanged'] = unchanged_rows

# Reset index for the final result
res.reset_index(drop=True, inplace=True)

In [8]:
res

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,date_stamp,last_change,new_entry,deletion,unchanged
0,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-07-12 13:52:13,20220712,NaN,True,False,False
1,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-07-13 18:12:14,20220713,NaN,False,False,True
2,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-07-14 18:12:16,20220714,NaN,False,False,True
3,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-07-15 18:13:21,20220715,NaN,False,False,True
4,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-07-16 18:11:37,20220716,NaN,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55478,eu-sancmap-febf18c1754d6a3242b40757aabb5937ad0...,Vessel,JIE SHUN,NaN,NaN,NaN,NaN,514569000;8518780,"""""",NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-08-12T18:02:21,20230812,2023-04-20T12:14:15,False,False,True
55479,eu-sancmap-febf18c1754d6a3242b40757aabb5937ad0...,Vessel,JIE SHUN,NaN,NaN,NaN,NaN,514569000;8518780,"""""",NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-08-13T18:03:17,20230813,2023-04-20T12:14:15,False,False,True
55480,eu-sancmap-febf18c1754d6a3242b40757aabb5937ad0...,Vessel,JIE SHUN,NaN,NaN,NaN,NaN,514569000;8518780,"""""",NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-08-14T18:02:57,20230814,2023-04-20T12:14:15,False,False,True
55481,eu-sancmap-febf18c1754d6a3242b40757aabb5937ad0...,Vessel,JIE SHUN,NaN,NaN,NaN,NaN,514569000;8518780,"""""",NaN,NaN,EU Sanctions Map,2023-04-20T12:14:15,2023-08-15T18:02:46,20230815,2023-04-20T12:14:15,False,False,True


In [9]:
#test thematching function
test = res.query('new_entry == True')
test

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,date_stamp,last_change,new_entry,deletion,unchanged
0,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-07-12 13:52:13,20220712,NaN,True,False,False
10,NK-32NYYLd4iT2JDc2pukDRqY,LegalEntity,Bank Dabrabyt,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False
39,NK-3TSuSKtWtcuB3qBdLC8Y8d,Vessel,UL JI BONG 6,NaN,NaN,NaN,NaN,9114555,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False
639,NK-4C4LEUKsLKwUnJmGeJjnp2,LegalEntity,ROSNEFT,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False
1239,NK-4JYGfcjP3Frn24sYQ3ZXE2,Vessel,PAEK MA,NaN,NaN,NaN,NaN,9066978,DPRK vessel M/V PAEK MA was involved in ship-t...,NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53359,eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...,LegalEntity,Pervyi Kanal,NaN,NaN,NaN,NaN,NaN,"""""",NaN,NaN,EU Sanctions Map,2023-03-09 12:40:12,2023-03-09 18:41:04,20230309,NaN,True,True,False
53360,eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...,Vessel,TONG SAN 2,NaN,NaN,NaN,NaN,445539000;8937675,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False
53820,eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...,Vessel,PETREL 8,NaN,NaN,NaN,NaN,620233000;9562233,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False
54420,eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...,Vessel,AN SAN 1 Other information: DPRK tanker M/V AN...,NaN,NaN,NaN,NaN,7303803,"""""",NaN,NaN,EU Sanctions Map,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False


# Deduplicate

In [10]:
res_deduplicated = res.copy()
res_deduplicated.sort_values(by=['id', 'date_stamp'], inplace=True)

# Identify new entries and deletions based on the index and columns you want to compare
new_entries = ~res_deduplicated.duplicated(subset=['id'], keep='first')
deletions = ~res_deduplicated.duplicated(subset=['id'], keep='last')

# Identify unchanged rows by checking for duplicates based on the id and date_stamp
unchanged_rows = ~new_entries & ~deletions

# Create new columns for first seen and last seen dates
res_deduplicated['listing_date'] = res_deduplicated['date_stamp'].where(new_entries)
res_deduplicated['delisting_date'] = res_deduplicated['date_stamp'].where(deletions)

# Add a 'month' column
res_deduplicated['month'] = pd.to_datetime(res_deduplicated['date_stamp']).dt.to_period('M')

# Deduplicate the entries (keep the first occurrence for each entity)
res_deduplicated = res_deduplicated.drop_duplicates(subset=['id'], keep='first')

# Reset index for the final result
res_deduplicated.reset_index(drop=True, inplace=True)

# Print or further analyze the deduplicated DataFrame 'res_deduplicated'
print(res_deduplicated)

                                                    id       schema  \
0                            NK-2kLa63Qa2yXPxpd3zCFrwt  LegalEntity   
1                            NK-32NYYLd4iT2JDc2pukDRqY  LegalEntity   
2                            NK-3TSuSKtWtcuB3qBdLC8Y8d       Vessel   
3                            NK-4C4LEUKsLKwUnJmGeJjnp2  LegalEntity   
4                            NK-4JYGfcjP3Frn24sYQ3ZXE2       Vessel   
..                                                 ...          ...   
129  eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...  LegalEntity   
130  eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...       Vessel   
131  eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...       Vessel   
132  eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...       Vessel   
133  eu-sancmap-febf18c1754d6a3242b40757aabb5937ad0...       Vessel   

                                                  name aliases birth_date  \
0                                           Sovcombank     NaN        

In [11]:
res_deduplicated

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,...,first_seen,last_seen,date_stamp,last_change,new_entry,deletion,unchanged,listing_date,delisting_date,month
0,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2022-05-10 07:48:17,2022-07-12 13:52:13,20220712,NaN,True,False,False,20220712,NaN,2022-07
1,NK-32NYYLd4iT2JDc2pukDRqY,LegalEntity,Bank Dabrabyt,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05
2,NK-3TSuSKtWtcuB3qBdLC8Y8d,Vessel,UL JI BONG 6,NaN,NaN,NaN,NaN,9114555,"""""",NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05
3,NK-4C4LEUKsLKwUnJmGeJjnp2,LegalEntity,ROSNEFT,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05
4,NK-4JYGfcjP3Frn24sYQ3ZXE2,Vessel,PAEK MA,NaN,NaN,NaN,NaN,9066978,DPRK vessel M/V PAEK MA was involved in ship-t...,NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...,LegalEntity,Pervyi Kanal,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2023-03-09 12:40:12,2023-03-09 18:41:04,20230309,NaN,True,True,False,20230309,20230309,2023-03
130,eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...,Vessel,TONG SAN 2,NaN,NaN,NaN,NaN,445539000;8937675,"""""",NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05
131,eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...,Vessel,PETREL 8,NaN,NaN,NaN,NaN,620233000;9562233,"""""",NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05
132,eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...,Vessel,AN SAN 1 Other information: DPRK tanker M/V AN...,NaN,NaN,NaN,NaN,7303803,"""""",NaN,...,2022-05-10 07:48:17,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05


# Descriptives and clean up

In [12]:
#the countries are coded as iso2 codes -> transform to iso 3
#keep both countries? could search for "ru" over regex functions
print(res_deduplicated.countries.unique())
print(res_deduplicated.schema.unique())
print(res_deduplicated.listing_date.unique())

[nan]
['LegalEntity' 'Vessel' 'Person']
['20220712' '20220510' '20230310' '20220620' '20220624' '20221109'
 '20230705' '20220702' '20230814' '20220608' '20220718' '20230817'
 '20220722' '20230421' '20220705' '20220520' '20220614' '20230309'
 '20220617' '20230704']


In [13]:
list_regimes = list[res_deduplicated.sanctions.unique()]
print(list_regimes)
#but it doesnt show me all

list[array(['""',
       'DPRK vessel M/V PAEK MA was involved in ship-to-ship transfer operations for oil in mid-January 2018.',
       'M/V DONG FENG 6 loaded DPRK coal at Hamhung, DPRK, on 11 July 2017, for export in violation of UN sanctions.',
       'M/V HUA FU loaded DPRK coal at Najin, DPRK, on 24 September 2017.',
       'M/V HAO FAN 2 loaded DPRK coal at Nampo, DPRK, on 3 June 2017, for export in violation of UN sanctions.',
       'M/V KOTI engaged in a ship-to-ship transfer, likely for oil, with M/V KUM UN SAN 3 on 9\xa0December 2017.',
       ';M/V HAO FAN 6 loaded DPRK coal at Nampo, DPRK, on 27 August 2017.',
       'DPRK oil tanker M/V SAM MA 2 imported refined petroleum products in October, early November and mid-November 2017 through multiple ship-to-ship transfers.',
       'The merchant vessel M/V SHANG YUAN BAO engaged in a ship-to-ship transfer, likely for oil, with UN-designated DPRK vessel M/V PAEK MA on May 18, 2018. The SHANG YUAN BAO also engaged in a ship-to

In [14]:
#get missingness
is_null = res_deduplicated.isnull().sum()
display(is_null)

id                  0
schema              0
name                0
aliases           118
birth_date        134
countries         134
addresses         134
identifiers        93
sanctions           0
phones            134
emails            134
dataset             0
first_seen          0
last_seen           0
date_stamp          0
last_change       124
new_entry           0
deletion            0
unchanged           0
listing_date        0
delisting_date    127
month               0
dtype: int64

In [15]:
res_deduplicated.columns

Index(['id', 'schema', 'name', 'aliases', 'birth_date', 'countries',
       'addresses', 'identifiers', 'sanctions', 'phones', 'emails', 'dataset',
       'first_seen', 'last_seen', 'date_stamp', 'last_change', 'new_entry',
       'deletion', 'unchanged', 'listing_date', 'delisting_date', 'month'],
      dtype='object')

In [16]:
res_deduplicated['new_listing_count'] = 0  # Initialize the 'listing_count' column with zeros
res_deduplicated['new_listing_count'] = res_deduplicated.groupby('month')['id'].transform('nunique')
res_deduplicated

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,...,last_seen,date_stamp,last_change,new_entry,deletion,unchanged,listing_date,delisting_date,month,new_listing_count
0,NK-2kLa63Qa2yXPxpd3zCFrwt,LegalEntity,Sovcombank,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2022-07-12 13:52:13,20220712,NaN,True,False,False,20220712,NaN,2022-07,12
1,NK-32NYYLd4iT2JDc2pukDRqY,LegalEntity,Bank Dabrabyt,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83
2,NK-3TSuSKtWtcuB3qBdLC8Y8d,Vessel,UL JI BONG 6,NaN,NaN,NaN,NaN,9114555,"""""",NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83
3,NK-4C4LEUKsLKwUnJmGeJjnp2,LegalEntity,ROSNEFT,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83
4,NK-4JYGfcjP3Frn24sYQ3ZXE2,Vessel,PAEK MA,NaN,NaN,NaN,NaN,9066978,DPRK vessel M/V PAEK MA was involved in ship-t...,NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,eu-sancmap-e51b08db944318e41b5d48fcea3d1e643ec...,LegalEntity,Pervyi Kanal,NaN,NaN,NaN,NaN,NaN,"""""",NaN,...,2023-03-09 18:41:04,20230309,NaN,True,True,False,20230309,20230309,2023-03,10
130,eu-sancmap-f19cf43dd475a50ac0b6eb94fe005ed95ad...,Vessel,TONG SAN 2,NaN,NaN,NaN,NaN,445539000;8937675,"""""",NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83
131,eu-sancmap-fd1013b334b2efa28ab2ceda6fc53fc6bc2...,Vessel,PETREL 8,NaN,NaN,NaN,NaN,620233000;9562233,"""""",NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83
132,eu-sancmap-feb60d88b4a9c985444b47b0c297cba9ea4...,Vessel,AN SAN 1 Other information: DPRK tanker M/V AN...,NaN,NaN,NaN,NaN,7303803,"""""",NaN,...,2022-05-10 18:12:58,20220510,NaN,True,False,False,20220510,NaN,2022-05,83


In [17]:
res_deduplicated = res_deduplicated.drop(['last_seen', 'first_seen','new_entry', 'deletion','unchanged'],axis = 1)

In [18]:
res_deduplicated.to_csv("eu_map.csv")